In [33]:
!pip install catboost

In [34]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score
import numpy as np
import joblib

In [35]:
train_dir =  r'D:\Subject\Computer Vision\Sections\Alzheimer_s Dataset\Alzheimer_s Dataset\train' 
test_dir =  r'D:\Subject\Computer Vision\Sections\Alzheimer_s Dataset\Alzheimer_s Dataset\test'

In [36]:
# Set the input shape
input_shape = (128, 128, 3)
batch_size = 32

In [37]:
# Use ImageDataGenerator for data augmentation
data_generator = ImageDataGenerator(rescale=1./255)

In [38]:
# Load the training images
train_generator = data_generator.flow_from_directory(
    train_dir,
    target_size=(input_shape[0], input_shape[1]),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)

Found 5121 images belonging to 4 classes.


In [39]:
# Load the test images
test_generator = data_generator.flow_from_directory(
    test_dir,
    target_size=(input_shape[0], input_shape[1]),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)

Found 1279 images belonging to 4 classes.


In [40]:
# Define the model architecture up to the feature extraction layer
model = Sequential([
    Conv2D(32, (3, 3), input_shape=input_shape),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3)),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(64),
    Activation('relu'),
    Dropout(0.5)
])

C:\Users\clt\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [41]:
# Extract features from training images 
train_features = model.predict(train_generator)
train_features = train_features.reshape(train_features.shape[0], -1)  # Flatten features for classifier
train_labels = train_generator.classes

C:\Users\clt\AppData\Roaming\Python\Python312\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


161/161 ━━━━━━━━━━━━━━━━━━━━ 14s 83ms/step


In [42]:
# Extract features from test images 
test_features = model.predict(test_generator)
test_features = test_features.reshape(test_features.shape[0], -1)
test_labels = test_generator.classes

40/40 ━━━━━━━━━━━━━━━━━━━━ 4s 98ms/step


In [51]:
# Use CatBoost classifier
cb_classifier = CatBoostClassifier(iterations=1000, learning_rate=0.1, depth=6)
cb_classifier.fit(train_features, train_labels)

0:	learn: 1.3244488	total: 53.7ms	remaining: 53.6s
1:	learn: 1.2730372	total: 96.7ms	remaining: 48.3s
2:	learn: 1.2276651	total: 146ms	remaining: 48.5s
3:	learn: 1.1912363	total: 174ms	remaining: 43.3s
4:	learn: 1.1564405	total: 205ms	remaining: 40.8s
5:	learn: 1.1272928	total: 237ms	remaining: 39.3s
6:	learn: 1.1006587	total: 271ms	remaining: 38.5s
7:	learn: 1.0786551	total: 297ms	remaining: 36.8s
8:	learn: 1.0583757	total: 323ms	remaining: 35.5s
9:	learn: 1.0392035	total: 346ms	remaining: 34.2s
10:	learn: 1.0238062	total: 367ms	remaining: 33s
11:	learn: 1.0084373	total: 396ms	remaining: 32.6s
12:	learn: 0.9951751	total: 428ms	remaining: 32.5s
13:	learn: 0.9823923	total: 459ms	remaining: 32.3s
14:	learn: 0.9706742	total: 491ms	remaining: 32.2s
15:	learn: 0.9606238	total: 521ms	remaining: 32.1s
16:	learn: 0.9498795	total: 555ms	remaining: 32.1s
17:	learn: 0.9402857	total: 581ms	remaining: 31.7s
18:	learn: 0.9321818	total: 607ms	remaining: 31.3s
19:	learn: 0.9254190	total: 636ms	remaini

In [44]:
# Make predictions on the test data
test_predictions = cb_classifier.predict(test_features)

In [ ]:
# Calculate accuracy
test_accuracy = accuracy_score(test_labels, test_predictions)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

In [46]:
# Save the Keras CNN model as an .h5 file
model.save('cnn_feature_extractor6.h5')

In [47]:
# Save the trained CatBoost model as a .pkl file
joblib.dump(cb_classifier, 'catboost_classifier.pkl')

['catboost_classifier.pkl']

In [53]:
from keras.models import load_model
import joblib
import numpy as np
from tensorflow.keras.preprocessing import image
from catboost import CatBoostClassifier

# Load the saved models
cnn_model = load_model("D:/Computer Vision Tasks/Sec4/CATBOOST/cnn_feature_extractor6.h5")
catboost_classifier = joblib.load("D:/Computer Vision Tasks/Sec4/CATBOOST/catboost_classifier.pkl")  # Load CatBoost model

def classify_image(img_path):
    # Load and preprocess the new image
    img = image.load_img(img_path, target_size=(128, 128))  # Resize to match the CNN input
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.0  # Normalize to match the training preprocessing

    # Extract features using the CNN model
    features = cnn_model.predict(img_array)
    features = features.reshape(1, -1)  # Flatten to 1D for CatBoost

    # Classify the features with the CatBoost model
    prediction = catboost_classifier.predict(features)
    
    # Ensure prediction is an integer
    predicted_class = int(prediction[0])

    # Interpret the prediction
    class_labels = {0: 'MildDemented', 1: 'ModerateDemented', 2: 'NonDemented', 3: 'VeryMildDemented'}  # Adjust based on your dataset labels
    result = class_labels[predicted_class]
    return result

# Test the function with a new image
img_path =  r'D:\Subject\Computer Vision\Sections\Alzheimer_s Dataset\Alzheimer_s Dataset\test\NonDemented\31 (77).jpg'
result = classify_image(img_path)
print(f"The image is classified as: {result}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 221ms/step
The image is classified as: VeryMildDemented


C:\Users\clt\AppData\Local\Temp\ipykernel_26644\147718814.py:26: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  predicted_class = int(prediction[0])
